In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Persiapan Dataset

Kl ngunain colab dan google drive, 
1. masukin csv dulu di google drive..trus mounting.
2. sesuain path csvnya

In [40]:
import pandas as pd

movies_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')
print() 

Potong Data cuma 100, biar cepet aja...

In [41]:
movies_df_100 = movies_df[:100]
ratings_df_100 = ratings_df[:100]
# movies_df_100
ratings_df_100

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
95,2,733,4.5,1141415905
96,2,858,3.5,1141416926
97,2,914,4.0,1141417642
98,2,953,4.5,1141417487


## Preprocessing

Ini gw nyesuain dengan format data asli kalau menggunakan tfds...

In [42]:
movies_df_100.movieId = movies_df_100.movieId.apply(lambda x: str(x))
movies_df_100.head()
print(type(movies_df_100.movieId[0]))

<class 'str'>


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
ratings_df_100.userId = ratings_df_100.userId.apply(lambda x: str(x)) 
ratings_df_100.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [44]:
ratings_df_100.movieId = ratings_df_100.movieId.apply(lambda x: str(x)) 
ratings_df_100.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


Biar mudah diproses, data "genre" disesuain formatnya biar jadi list

In [46]:
movies_df_100["genres"] = movies_df_100["genres"].apply(lambda x: x.split("|"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
movies_df_100

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
95,97,"Hate (Haine, La) (1995)","[Crime, Drama]"
96,98,Shopping (1994),"[Action, Thriller]"
97,99,Heidi Fleiss: Hollywood Madam (1995),[Documentary]
98,100,City Hall (1996),"[Drama, Thriller]"


In [48]:
unique_genre = []
def getGenres(datas,column_name):
  unique_genre = set()
  for x in datas[column_name]:
    unique_genre.update(x)
  return sorted(list(unique_genre))

unique_genre = getGenres(movies_df_100,"genres")
print(unique_genre)

string_look_up = tf.keras.layers.StringLookup(vocabulary=unique_genre)

['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War']


## One Hot Encoding

In [49]:
print(list(movies_df_100.genres))

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], ['Adventure', 'Children', 'Fantasy'], ['Comedy', 'Romance'], ['Comedy', 'Drama', 'Romance'], ['Comedy'], ['Action', 'Crime', 'Thriller'], ['Comedy', 'Romance'], ['Adventure', 'Children'], ['Action'], ['Action', 'Adventure', 'Thriller'], ['Comedy', 'Drama', 'Romance'], ['Comedy', 'Horror'], ['Adventure', 'Animation', 'Children'], ['Drama'], ['Action', 'Adventure', 'Romance'], ['Crime', 'Drama'], ['Drama', 'Romance'], ['Comedy'], ['Comedy'], ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller'], ['Comedy', 'Crime', 'Thriller'], ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'], ['Action', 'Crime', 'Thriller'], ['Drama', 'Sci-Fi'], ['Drama', 'Romance'], ['Drama'], ['Children', 'Drama'], ['Drama', 'Romance'], ['Adventure', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi'], ['Crime', 'Drama'], ['Drama'], ['Mystery', 'Sci-Fi', 'Thriller'], ['Adventure', 'Romance', 'IMAX'], ['Children', 'Drama'], ['Drama', 'Romance'], ['Crime', 'Drama'], ['D

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer

features = list(movies_df_100.genres)
mlb = MultiLabelBinarizer()
new_features = mlb.fit_transform(features)
print(type(new_features))

<class 'numpy.ndarray'>


In [51]:
print(mlb.inverse_transform(new_features))

[('Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'), ('Adventure', 'Children', 'Fantasy'), ('Comedy', 'Romance'), ('Comedy', 'Drama', 'Romance'), ('Comedy',), ('Action', 'Crime', 'Thriller'), ('Comedy', 'Romance'), ('Adventure', 'Children'), ('Action',), ('Action', 'Adventure', 'Thriller'), ('Comedy', 'Drama', 'Romance'), ('Comedy', 'Horror'), ('Adventure', 'Animation', 'Children'), ('Drama',), ('Action', 'Adventure', 'Romance'), ('Crime', 'Drama'), ('Drama', 'Romance'), ('Comedy',), ('Comedy',), ('Action', 'Comedy', 'Crime', 'Drama', 'Thriller'), ('Comedy', 'Crime', 'Thriller'), ('Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'), ('Action', 'Crime', 'Thriller'), ('Drama', 'Sci-Fi'), ('Drama', 'Romance'), ('Drama',), ('Children', 'Drama'), ('Drama', 'Romance'), ('Adventure', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi'), ('Crime', 'Drama'), ('Drama',), ('Mystery', 'Sci-Fi', 'Thriller'), ('Adventure', 'IMAX', 'Romance'), ('Children', 'Drama'), ('Drama', 'Romance'), ('Crime', 'Drama

In [52]:
movies_from_csv = []
# df100 = df100.reset_index()
movie_genres = []
movie_ids = []
movie_titles = []
for index,x in movies_df_100.iterrows():
  # print(x["genres"])
  movie_genres.append(tf.constant(new_features))
  movie_ids.append(tf.constant(value = x["movieId"],dtype = tf.string))
  movie_titles.append(tf.constant(x["title"]))

#   movies_from_csv.append({
#       "movie_genres": tf.constant(new_features),
#       "movie_id": tf.constant(value = x["movieId"],dtype = tf.string),
#       "movie_title": tf.constant(x["title"])
#   })

# pprint.pprint(movies_from_csv[:3])
print(movie_genres[:5])

[<tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])>, <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])>, <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])>, <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0,

In [53]:

m_i = []
t_s = []
u_i = []
u_r = []
# userId	movieId	rating	timestamp

for index,x in ratings_df_100.iterrows():
  m_i.append(tf.constant(x["movieId"]))
  t_s.append(tf.constant(x["timestamp"], dtype = np.int64))
  u_i.append(tf.constant(x["userId"]))
  u_r.append(tf.constant(x["rating"]))
print(m_i[:3])
print(t_s[:3])
print(u_i[:3])
print(u_r[:3])

[<tf.Tensor: shape=(), dtype=string, numpy=b'296'>, <tf.Tensor: shape=(), dtype=string, numpy=b'306'>, <tf.Tensor: shape=(), dtype=string, numpy=b'307'>]
[<tf.Tensor: shape=(), dtype=int64, numpy=1147880044>, <tf.Tensor: shape=(), dtype=int64, numpy=1147868817>, <tf.Tensor: shape=(), dtype=int64, numpy=1147868828>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'1'>, <tf.Tensor: shape=(), dtype=string, numpy=b'1'>, <tf.Tensor: shape=(), dtype=string, numpy=b'1'>]
[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=3.5>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]


Ini yang jadi penyama format dataset yang dari tfds ataupun custom dari csv

In [54]:
movies = tf.data.Dataset.from_tensor_slices({"movie_genres":movie_genres ,"movie_id": movie_ids, "movie_title":movie_titles}) 
ratings = tf.data.Dataset.from_tensor_slices({"movieId":m_i ,"timestamp": t_s, "userId":u_i, "rating":u_r}) 

In [55]:
for i in movies.take(3):
  print(i)
print()
for i in ratings.take(3):
  print(i)

{'movie_genres': <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Toy Story (1995)'>}
{'movie_genres': <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'2'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Jumanji (1995)'>}
{'movie_genres': <tf.Tensor: shape=(100, 17), dtype=int64, numpy=
array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
  

## Start Extract Main Data

Intinya, di retrieval model, model ngebuat matrix fitur gitu... 
Ada 2 bagian di contohnya(bisa disesuain nantinya):
- query tower
- candidate tower


yang bagian kolomnya akan berisi userId, yang bagian row rownya akan bernilai movieID ataupun movieTitle.

Karna dataset sedikit berbeda antara yang movielens100k dan movielens25M...gw gunain data yang sama, yaitu movieId... 

seharusnya sih movieTitle

tapi gamasalah

In [56]:
movies = movies.map(lambda x: x["movie_id"])
ratings = ratings.map(lambda x: {
    "movieId": x["movieId"],
    "userId": x["userId"],
})

In [57]:
for i in ratings.take(3):
  print(i)
for i in movies.take(3):
  print(i)
print(len(list(ratings.as_numpy_iterator())))

{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'296'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'306'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'307'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
tf.Tensor(b'1', shape=(), dtype=string)
tf.Tensor(b'2', shape=(), dtype=string)
tf.Tensor(b'3', shape=(), dtype=string)
100


In [58]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80)
test = shuffled.skip(80).take(20)

In [59]:
for i in train.take(3):
  print(i)

{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'5767'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'7327'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}
{'movieId': <tf.Tensor: shape=(), dtype=string, numpy=b'5878'>, 'userId': <tf.Tensor: shape=(), dtype=string, numpy=b'1'>}


In [60]:
movie_titles = movies.batch(10)
user_ids = ratings.batch(10).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b'1', b'10', b'100', b'101', b'11', b'12', b'13', b'14', b'15',
       b'16'], dtype=object)

## Query TOwer

In [61]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

## Candidate Tower

In [62]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [63]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [87]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    print(features)
    user_embeddings = self.user_model(features["userId"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movieId"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [88]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["userId"])
      positive_movie_embeddings = self.movie_model(features["movieId"])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["userId"])
    positive_movie_embeddings = self.movie_model(features["movieId"])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

## Fitting

In [89]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [90]:
cached_train = train.shuffle(100).batch(10).cache()
cached_test = test.batch(10).cache()

In [91]:
model.fit(cached_train, epochs=3)

Epoch 1/3
{'movieId': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'userId': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>}
{'movieId': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'userId': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>}
8/8 [==============================] - 2s 56ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0750 - factorized_top_k/top_5_categorical_accuracy: 0.0750 - factorized_top_k/top_10_categorical_accuracy: 0.0750 - factorized_top_k/top_50_categorical_accuracy: 0.1000 - factorized_top_k/top_100_categorical_accuracy: 0.9500 - loss: 23.0246 - regularization_loss: 0.0000e+00 - total_loss: 23.0246
Epoch 2/3
8/8 [==============================] - 0s 52ms/step - factorized_top_k/top_1_categorical_accuracy: 0.6625 - factorized_top_k/top_5_categorical_accuracy: 0.6875 - factorized_top_k/top_10_categorical_accuracy: 0.6875 - factorized_top_k/top_50_categorical_accuracy: 0.6875 - factorized_top_k/top_100_c

In [92]:
model.evaluate(cached_test, return_dict=True)

{'movieId': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'userId': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>}
2/2 [==============================] - 1s 56ms/step - factorized_top_k/top_1_categorical_accuracy: 0.8500 - factorized_top_k/top_5_categorical_accuracy: 0.8500 - factorized_top_k/top_10_categorical_accuracy: 0.8500 - factorized_top_k/top_50_categorical_accuracy: 0.8500 - factorized_top_k/top_100_categorical_accuracy: 0.8500 - loss: 23.0259 - regularization_loss: 0.0000e+00 - total_loss: 23.0259


{'factorized_top_k/top_100_categorical_accuracy': 0.8500000238418579,
 'factorized_top_k/top_10_categorical_accuracy': 0.8500000238418579,
 'factorized_top_k/top_1_categorical_accuracy': 0.8500000238418579,
 'factorized_top_k/top_50_categorical_accuracy': 0.8500000238418579,
 'factorized_top_k/top_5_categorical_accuracy': 0.8500000238418579,
 'loss': 23.02585220336914,
 'regularization_loss': 0,
 'total_loss': 23.02585220336914}

In [99]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
no_user = "1"
jumlah_rekomendasi = 10

_, titles = index(tf.constant([no_user]))
print(f"Recommendations for user{no_user}: {titles[0, :jumlah_rekomendasi]}")

Recommendations for user1: [b'70' b'9' b'87' b'57' b'20' b'86' b'92' b'25' b'56' b'74']
